# <font color='blue'>Projeto com Grafos - Transferências de Jogadores</font>

## <font color='blue'>Transferência de Jogadores em grafos - Estudo</font>


In [62]:
#Importando Pacotes
import pandas as pd
import sqlite3
import numpy as np
!pip -q install mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
!pip -q install pyvis
from pyvis import network as net
import networkx as nx


##Pre sets##

#Maximo Colunas
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [63]:
# Extraindo as tabelas do .sqlite
con = sqlite3.connect("Arquivo - Soccer/database.sqlite")
players = pd.read_sql_query("SELECT * FROM Player", con)
players_stats = pd.read_sql_query("SELECT * FROM Player_Attributes", con)
teams = pd.read_sql_query("SELECT * FROM Team", con)
league = pd.read_sql_query("SELECT * FROM League", con)
Matches = pd.read_sql_query("SELECT * FROM Match", con)

con.close()

In [64]:
#Substituir caracteres e espaço por "_" no dataframe de Times

teams.team_long_name.replace(" ","_", regex=True, inplace=True)
teams.team_long_name.replace("-","_", regex=True, inplace=True)
teams.team_long_name.replace("\\.","_", regex=True, inplace=True)

In [65]:
#Merge de Matches, League e Teams

CountryClub = pd.merge_asof(Matches, league, o n="country_id", by='country_id')
CountryClub = CountryClub.merge(teams, how='left', left_on="home_team_api_id", right_on="team_api_id")

In [66]:
#Merge entre Jogadores e Times

#Transformar as colunas de jogadores em uma coluna unica
tmp = Matches.melt(id_vars=['season','home_team_api_id'], 
               value_vars=['home_player_1','home_player_2','home_player_3','home_player_4','home_player_5','home_player_6','home_player_7','home_player_8','home_player_9','home_player_10','home_player_11'],
                value_name='player_api_id')

#Excluir os valores nulos
tmp = tmp.dropna()

#Transformar os valores da planilha players em float
players = players.astype({"player_api_id": float,})

#Realizar o merge entre Matches e Players
left_sorted = tmp.sort_values(by="player_api_id")
right_sorted = players.sort_values(by="player_api_id")

tmp = pd.merge_asof(left_sorted, right_sorted, on="player_api_id")

#Realizar o merge entre Jogadores e Times
TeamsPlayer = tmp.merge(teams, how='left', left_on="home_team_api_id", right_on="team_api_id")

#Deixar em classificação alfabetica
TeamsPlayer = TeamsPlayer.sort_values('id_x')

#salvar em csv

TeamsPlayer.to_csv(r'C:\Users\Jefferson\OneDrive\Ciência de Dados\DSA\Big Data Analytics com R e Microsoft Azure Machine Learning\Manipulação de Arquivos TXT, CSV e Planilhas Excel\Projeto-Transacoes-no-Futebol\TeamsPlayer.txt', header=True, index=True, sep=' ', mode='a')



In [67]:
#Criando cesta de dados

basket = (TeamsPlayer
          .groupby(['player_name', 'team_long_name'])['id_x']
          .sum().unstack().reset_index().fillna(0)
          .set_index('player_name'))

#Transformando informações em binário para utilização do apriori
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

#Criando dataset de frequencia
frequent_itemsets = apriori(basket_sets, min_support=0.00010, use_colnames=True)

#Criando associação dos dados da cesta
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [71]:
rules.sort_values("support")

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3176,(KV_Kortrijk),(KSV_Roeselare),0.007538,0.002513,0.000193,0.025641,10.204142,0.000174,1.023737
4310,(Sevilla_FC),(Real_Zaragoza),0.009181,0.006282,0.000193,0.021053,3.351255,0.000136,1.015088
4309,(Real_Zaragoza),(SL_Benfica),0.006282,0.008408,0.000193,0.030769,3.659416,0.000140,1.023071
4308,(SL_Benfica),(Real_Zaragoza),0.008408,0.006282,0.000193,0.022989,3.659416,0.000140,1.017100
4303,(Real_Sporting_de_Gijón),(Sevilla_FC),0.005895,0.009181,0.000193,0.032787,3.571009,0.000139,1.024406
4302,(Sevilla_FC),(Real_Sporting_de_Gijón),0.009181,0.005895,0.000193,0.021053,3.571009,0.000139,1.015483
4295,(Real_Madrid_CF),(SC_Braga),0.006185,0.010341,0.000193,0.031250,3.021904,0.000129,1.021583
4294,(SC_Braga),(Real_Madrid_CF),0.010341,0.006185,0.000193,0.018692,3.021904,0.000129,1.012744
4293,(Roma),(Real_Madrid_CF),0.009085,0.006185,0.000193,0.021277,3.439827,0.000137,1.015419
4292,(Real_Madrid_CF),(Roma),0.006185,0.009085,0.000193,0.031250,3.439827,0.000137,1.022880
